In [50]:
import pandas as pd

from zenodo_parser import ZenodoParser

parser = ZenodoParser()

dat = parser.load("pickle_test_all_with_files_sparse.pickle").data

print('dense : {:0.2f} bytes'.format(dat.memory_usage().sum() / 1e3))


dense : 52867.64 bytes


In [54]:
pd.set_option('display.max_seq_items', None)

Index(['conceptrecid', 'created', 'doi', 'files', 'id', 'links', 'owners',
       'revision', 'stats', 'updated', 'access_right', 'access_right_category',
       'communities', 'creators', 'description', 'keywords', 'license',
       'locations', 'notes', 'publication_date', 'related_identifiers',
       'relations', 'resource_type', 'title', 'csv_count', 'csv_paths',
       'txt_count', 'txt_paths', 'csv_mean_size', 'csv_std_size',
       'txt_mean_size', 'txt_std_size', 'download_time', 'zip_count',
       'zip_paths', 'zip_mean_size', 'zip_std_size', 'nex_count', 'nex_paths',
       'xlsx_count', 'xlsx_paths', 'pdf_count', 'pdf_paths', 'pdf_mean_size',
       'pdf_std_size', 'nex_mean_size', 'nex_std_size', 'xlsx_mean_size',
       'xlsx_std_size', 'xls_count', 'xls_paths', 'docx_count', 'docx_paths',
       'doc_count', 'doc_paths', 'doc_mean_size', 'doc_std_size',
       'docx_mean_size', 'docx_std_size', 'xls_mean_size', 'xls_std_size',
       'rtf_count', 'rtf_paths', 'rtf_mean_

In [56]:
dat.creators

0       [{'affiliation': 'Brown University', 'name': '...
1       [{'affiliation': 'Imperial College London', 'n...
2       [{'affiliation': 'Hokkaido University', 'name'...
3       [{'affiliation': 'Imperial College London', 'n...
4       [{'affiliation': 'California Academy of Scienc...
                              ...                        
2901    [{'affiliation': 'Centre for Isotope Research ...
2902    [{'name': 'Raji, Shahab', 'orcid': '0000-0002-...
2903    [{'affiliation': 'University of Lodz, Doctoral...
2904    [{'affiliation': 'Department of Neurodegenerat...
2905    [{'name': 'Pengyu Chu', 'orcid': '0000-0002-82...
Name: creators, Length: 183380, dtype: object

In [ ]:
def to_sparse(x):
    if x.isna().sum() / len(x) > 0.3:
        return pd.arrays.SparseArray(x)
    return x

sparse_dat = dat.apply(to_sparse)

In [48]:
print("halted"[:3], "halted"[3:])

print('dense : {:0.2f} bytes'.format(dat.memory_usage().sum() / 1e3))

print('sparse: {:0.2f} bytes'.format(sparse_dat.memory_usage().sum() / 1e3))

print(sparse_dat.dtypes)
print(dat.isna().sum())

hal ted
dense : 64992.73 bytes
sparse: 40989.27 bytes
index                                  int64
conceptrecid                          object
created                               object
doi                                   object
files                                 object
id                                     int64
links                                 object
owners                                object
revision                               int64
stats                                 object
updated                               object
access_right                          object
access_right_category                 object
communities              Sparse[object, nan]
creators                              object
description                           object
keywords                 Sparse[object, nan]
license                               object
locations                Sparse[object, nan]
notes                    Sparse[object, nan]
publication_date                      object
r

In [27]:
import numpy as np
# print(dat.dates)
dtype = pd.SparseDtype(object, fill_value=np.nan)

# sparse_dates = dat.dates.astype(dtype)
# sparse_dates.sparse.density
sparse_all = dat.sparse
# sparse_all.sparse.density

AttributeError: Can only use the '.sparse' accessor with Sparse data.

In [24]:
# i = dat["filetypes"].loc[0]
# i

occurrence_count = dict()
occurrence_ratio = dict()
def count_occurrences(file_type_dict):
    sum_of_all = sum(file_type_dict.values())

    for key in file_type_dict:
        count = occurrence_count.get(key, 0)
        sum_of_all += file_type_dict[key]
        count += file_type_dict[key]
        occurrence_count[key] = count
        ratio = occurrence_ratio.get(key, 0)
        ratio += file_type_dict[key] / sum_of_all
        occurrence_ratio[key] = ratio



In [9]:
occurrence_count.values()

t = dat["filetypes"].apply(count_occurrences)

In [ ]:

def modify_frame(filetype_dict):
    for filetype, count in filetype_dict:
        filetypes.add(filetype)
        count = files_data.get(f"{filetype}_{COUNT}", 0)
        sizes = files_data.get(f"{filetype}_{SIZES}", [])
        paths = files_data.get(f"{filetype}_{PATHS}", [])
        count += 1
        sizes.append(file.get("size", 0))
        paths.append(file.get("links", dict()).get("self", ""))
        files_data[f"{filetype}_{COUNT}"] = int(count)
        files_data[f"{filetype}_{SIZES}"] = sizes
        files_data[f"{filetype}_{PATHS}"] = paths

In [10]:
import pandas as pd
sorted_occurrence_count = dict(sorted(list(occurrence_count.items()), key=lambda x: x[1], reverse=True))
df = pd.DataFrame.from_records([sorted_occurrence_count, occurrence_ratio], index=["occurrence_count", "occurrence_ratio"]).transpose()
df
# occurrence_count

,occurrence_count,occurrence_ratio
nc,626120.0,1284.265233
zip,142118.0,19198.428240
csv,129064.0,9593.060618
txt,123919.0,6235.630595
gz,121560.0,5967.594015
...,...,...
part-aj,1.0,0.021739
part-ak,1.0,0.021277
part-al,1.0,0.020833
topojson,1.0,0.025641


In [11]:
total_occurrence_count = sum(occurrence_count.values())
top160_filetypes = dict(sorted(list(occurrence_count.items()), key=lambda x: x[1], reverse=True)[:160])
print(top160_filetypes)
top160_occurrence_count = sum(top160_filetypes.values())
top160_occurrence_captured = top160_occurrence_count / total_occurrence_count
print(top160_occurrence_captured)

rest_filetypes = dict(sorted(list(occurrence_count.items()), key=lambda x: x[1], reverse=True)[160:])
print(rest_filetypes)
rest_occurrence_count = sum(rest_filetypes.values())
rest_occurrence_captured = rest_occurrence_count / total_occurrence_count
rest_occurrence_captured

{'nc': 626120, 'zip': 142118, 'csv': 129064, 'txt': 123919, 'gz': 121560, 'tif': 108176, 'jpg': 97442, 'h5': 68754, 'png': 66940, 'xlsx': 62820, 'mat': 51209, 'rar': 48147, 'bz2': 35370, 'pdf': 31599, 'hdf5': 30088, 'wav': 25075, 'dat': 19379, 'cbf': 19321, 'hdf': 17888, 'json': 17296, '': 16959, 'raw': 14423, 'xz': 12658, 'docx': 12059, 'fasta': 11532, 'czi': 9120, 'out': 8762, 'geojson': 8402, 'r': 8379, 'asc': 8320, 'xml': 8279, 'tiff': 8264, 'xls': 8228, 'img': 7486, 'hdr': 7219, 'dcm': 6921, 'fits': 6359, 'npz': 6050, 'tsv': 6014, 'mp4': 5982, 'npy': 4949, 'tgz': 4946, 'tar': 4916, 'md': 4591, 'rds': 4584, 'pkl': 4479, 'cfit': 4415, 'ttl116447': 4399, 'cdf': 4304, 'mzml': 4277, '7z': 4239, 'pdb': 3979, 'fcs': 3881, 'm': 3810, 'dill': 3779, 'nex': 3693, 'html': 3395, 'sav': 3342, 'bam': 3166, 'xtc': 2943, 'dng': 2941, 'py': 2909, 'rdata': 2884, 'fas': 2837, 'mccd': 2789, 'org': 2776, 'he5': 2709, 'spe': 2705, 'tpr': 2580, 'tre': 2575, 'fastq': 2491, 'vcf': 2484, 'odt': 2354, 'ply':

0.9579507956811609

In [7]:
from seaborn import barplot, countplot
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(num=None, figsize=(50,200), dpi=256, facecolor='w', edgecolor='r')
# plt.title("Missing Value Prerceentag")
barplot(data=df, x="occurrence_count", y=df.index)
plt.show()
plt.savefig("plot.png")
# countplot(data=df["occurrence_count"])

KeyboardInterrupt: 

Error in callback <function _draw_all_if_interactive at 0x000001D0BB0D3BE0> (for post_execute):


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x000001D0B9C44E50> (for post_execute):



KeyboardInterrupt

